# Tutorial4: 大模型推理

本节旨在使用 [cpm-bee-2b](https://huggingface.co/openbmb/cpm-bee-2b) 模型展示大模型推理。

分以下几步来实现：
1. 环境安装
2. 下载模型
3. 模型推理

## 1. 环境安装

请确保已经执行了 [tutorial_scow_for_ai](../tutorial_scow_for_ai.md) 中的"安装依赖、注册ipykernel"。

建议使用1张910B NPU运行本教程。

## 2. 下载模型

在联网的命令行中执行以下命令，命令执行位置在当前文件所在的文件夹。

```bash
export HF_ENDPOINT=https://hf-mirror.com
huggingface-cli download --resume-download openbmb/cpm-bee-2b --local-dir cpm-bee-2b
```

## 3. 模型推理

CPM-Bee 是一个完全开源、可商用的中英双语基础模型，拥有一百亿参数的容量。我们这里选用的是较小参数的 cpm-bee-2b 模型，下面展示加载 cmp-bee-2b 模型进行中翻英任务并统计生成速度的过程。

推理过程中使用 npu-smi info 命令可以查看 NPU 运行情况。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import time
import torch_npu
# 模型路径
model_path = "cpm-bee-2b"

# 硬件
device = torch.device('npu:0') if torch.npu.is_available() \
    else torch.device('cpu')

# 分词器
tokenizer = AutoTokenizer.from_pretrained(
    model_path, trust_remote_code=True)

# 模型
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,
    trust_remote_code=True
).to(device)

# 生成
start_time = time.time()
res = model.generate(
    {"input": "NGC 6231是一个位于天蝎座的疏散星团，天球座标为赤经16时54分，"
     "赤纬-41度48分，视觉观测大小约45角分，亮度约2.6视星等，距地球5900光年。"
     "NGC 6231年龄约为三百二十万年，是一个非常年轻的星团，"
     "星团内的最亮星是5等的天蝎座 ζ1星。用双筒望远镜或小型望远镜就能看到个别的行星。"
     "NGC 6231在1654年被意大利天文学家乔瓦尼·巴蒂斯特·霍迪尔纳"
     "（Giovanni Battista Hodierna）以Luminosae的名字首次纪录在星表中，"
     "但是未见记载于夏尔·梅西耶的天体列表和威廉·赫歇尔的深空天体目录。"
     "这个天体在1678年被爱德蒙·哈雷（I.7）、1745年被夏西亚科斯"
     "（Jean-Phillippe Loys de Cheseaux）（9）、1751年被尼可拉·路易·拉卡伊（II.13）"
     "分别再次独立发现。", "prompt": "中翻英", "<ans>": ""},
    tokenizer,
    max_new_tokens=1000,
    min_length=50,
)
end_time = time.time()

# calculate speed
num_generated_tokens = 0
for output in res:
    text = output['<ans>']
    tokens = tokenizer(text, return_tensors='pt')["input_ids"]
    num_tokens = tokens.shape[1]
    num_generated_tokens += num_tokens
time_taken = end_time - start_time
tokens_per_second = num_generated_tokens / time_taken
print(f"Generated {num_generated_tokens} tokens in {time_taken:.2f}" +\
      f" seconds ({tokens_per_second:.2f} tokens/second)")

print(res[0]['<ans>'])

---

> 作者: 黎颖; 褚苙扬; 龙汀汀
>
> 联系方式: yingliclaire@pku.edu.cn; cly2412307718@stu.pku.edu.cn; l.tingting@pku.edu.cn